In [ ]:
"""
Author: Mitch Constable & A.V. Ronquillo
Date: May 17, 2024

Purpose: To create bounding boxes using resnet50 on dataset
of animal images.

Note: The author generated this text in part with GPT-4,
OpenAI’s large-scale language-generation model. Upon generating
draft code, the author reviewed, edited, and revised the code
to their own liking and takes ultimate responsibility for
the content of this code.

"""

# Introduction

This notebook uses a pre-trained Faster R-CNN model to automatically detect and locate objects in 100 deer images by generating bounding boxes with confidence scores. The workflow loads the model, processes images, runs inference, and filters results based on a configurable confidence threshold before saving annotated outputs.

# Critical Uses & Adaptability

## What the Notebook Can Be Used For:

- **Dataset Exploration:** The notebook enables exploration of image datasets by automatically detecting and highlighting objects of interest. This is useful for quickly assessing the content and quality of a dataset, identifying labeling inconsistencies, or verifying the presence of target objects (in our case urban animals).

- **Educational Purposes & Demonstrations:** The notebook shows how Python code, scripting, and machine learning models can be applied to image-based tasks. By extracting bounding box coordinates, class labels, and confidence scores, the structured data can be used for further analysis such as statistical summaries, dataset curation, or as input features for other machine learning models.

## How the Notebook Can Be Adapted:

- **Integration with Spatial Design:** The methodology can be extended to site analysis or spatial design projects by applying object detection to images of built environments, landscapes, or architectural sites. Detected features can inform spatial mapping, usage studies, or environmental assessments.

- **Variables & Customization:** Variables like `confidence_threshold` (see Cell 7) and `image_dir` (see Cell 6) can be adjusted to refine detection sensitivity or to point to different image sources. By changing the `image_dir` variable in the code block in Cell 6 you can process any directory of images.

- **Different Data Sources:** The Borealis API functions can be modified to work with other data repositories, or the image directory detection can be pointed to local or cloud storage locations.

- **Extended Analysis:** The detection results can be exported to CSV or JSON formats for further statistical analysis, or integrated with spatial analysis tools for geographic applications.

## Importing Essential Libraries & Environment Setup

This module imports all essential libraries for the complete pipeline. The key addition is the proper `FasterRCNN_ResNet50_FPN_Weights` import which fixes the deprecated `pretrained=True parameter` issue. This environment setup will also include the import of `math` for advanced visualization layouts and `shutil` for file operations.

In [1]:
import os
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torchvision.transforms import functional as F
from PIL import Image
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import requests
import zipfile
import shutil
import math

## Borealis API Configuration & Source Dataset Functions

The data is hosted in the University of Manitoba Dataverse (https://borealisdata.ca/dataverse/manitoba), a research data repository. The images used in this notebook were collected as part of the 'Understanding Animals' project at University of Manitoba Faculty of Architecture, online at [Wild Winnipeg](https://www.wildwinnipeg.org/) and [Teaching with Images](https://pressbooks.openedmb.ca/teachingwithimages/). These functions handle the automatic downloading of datasets from the Borealis repository. The `get_public_dataset_info()` function retrieves metadata about available files, while `download_public_file()` handles the actual file download with proper error handling and progress feedback.

In [ ]:
# Borealis API configuration
BOREALIS_SERVER = "https://borealisdata.ca"

def get_public_dataset_info(persistent_id):
    """
    Get information about a public dataset
    """
    url = f"{BOREALIS_SERVER}/api/datasets/:persistentId/"
    params = {"persistentId": persistent_id}

    response = requests.get(url, params=params)

    if response.status_code == 200:
        dataset_info = response.json()
    else:
        print(f"Cannot access dataset: {response.status_code}")
        return None

    # Get a list of files in a public dataset
    files_list = dataset_info['data']['latestVersion']['files']
    file_info_list = []

    for file_info in files_list:
        file_id = file_info['dataFile']['id']
        filename = file_info['dataFile']['filename']
        file_info_list.append({"file_id": file_id, "filename": filename})

    return file_info_list

def download_public_file(file_id, save_path="./"):
    """
    Download a specific public file from a dataset by its file ID
    """
    url = f"{BOREALIS_SERVER}/api/access/datafile/{file_id}"
    response = requests.get(url, stream=True)

    if response.status_code == 200:
        filename = None
        if "Content-Disposition" in response.headers:
            cd = response.headers["Content-Disposition"]
            if "filename=" in cd:
                filename = cd.split("filename=")[1].strip('"')

        if not filename:
            filename = url.split("/")[-1]

        file_path = os.path.join(save_path, filename)

        with open(file_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        print(f"SUCCESS: File downloaded to {file_path}")
        return file_path
    else:
        print(f"ERROR: {response.status_code}: File may be restricted or not found")
        return None

## File Extraction & Directory Management

This module provides robust file handling capabilities. The `unzip_file()` function safely extracts ZIP archives, while `find_image_directory()` intelligently searches through the extracted contents to locate the directory containing image files. This eliminates the need for manual path specification.

In [ ]:
def is_zip_file(filepath):
    """
    Checks if a file is a valid zip file.
    """
    return zipfile.is_zipfile(filepath)

def unzip_file(filepath, extract_path="./"):
    """
    Unzips a zip file to a specified path
    """
    if is_zip_file(filepath):
        try:
            with zipfile.ZipFile(filepath, 'r') as zip_ref:
                zip_ref.extractall(extract_path)
                print(f"SUCCESS: Successfully unzipped {filepath} to {extract_path}")

                # List extracted contents
                extracted_files = zip_ref.namelist()
                print(f"Extracted {len(extracted_files)} files")
                return True
        except Exception as e:
            print(f"ERROR: Error unzipping {filepath}: {e}")
            return False
    else:
        print(f"INFO: {filepath} is not a valid zip file.")
        return False

def find_image_directory():
    """
    Find where the images were extracted
    """
    # Look for common patterns
    possible_dirs = []

    # Check current directory for any folders containing images
    for item in os.listdir('.'):
        if os.path.isdir(item):
            # Check if this directory contains JPG files
            jpg_files = glob.glob(os.path.join(item, '**/*.JPG'), recursive=True)
            if jpg_files:
                possible_dirs.append((item, len(jpg_files)))

    if possible_dirs:
        # Sort by number of JPG files found
        possible_dirs.sort(key=lambda x: x[1], reverse=True)
        best_dir = possible_dirs[0][0]

        # Find the actual subdirectory with images
        for root, dirs, files in os.walk(best_dir):
            jpg_files = [f for f in files if f.endswith('.JPG')]
            if jpg_files:
                print(f"📁 Found {len(jpg_files)} images in: {root}")
                return root

    return None

## Output Folder Creation & Organization

This short module creates a structured output directory system. The main `animal_detection_results` folder contains a subfolder for `annotated_images`, providing clear organization of results. The `exist_ok=True` parameter prevents errors if directories already exist.

In [ ]:
def create_output_folders():
    """
    Create organized output folders
    """
    # Create main output directory
    output_dir = "animal_detection_results"
    annotated_dir = os.path.join(output_dir, "annotated_images")

    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(annotated_dir, exist_ok=True)

    print(f"✅ Created output directory: {output_dir}")
    print(f"✅ Created annotated images directory: {annotated_dir}")

    return output_dir, annotated_dir

## Load the Model with Proper COCO Weights

In this critical module, instead of using the deprecated `pretrained=True parameter`, the `weights=FasterRCNN_ResNet50_FPN_Weights` will now be integrated into the notebook script. Then, the `COCO_V1` properly loads the COCO-trained weights. The model is set to evaluation mode for inference, and output directories are created immediately after model loading.

In [ ]:
# Load the pretrained model with proper weights parameter
print("📥 Loading pre-trained model...")
model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1)
model = model.eval()
print("✅ Model loaded successfully")

# Create output directories
output_dir, annotated_dir = create_output_folders()

## Dataset Download & Extraction

This module handles the complete dataset acquisition process. It connects to the Borealis repository using the specified DOI, downloads the ZIP file containing the animal images, extracts it, and then intelligently locates the directory containing the actual image files. The robust error handling ensures the process fails gracefully if any step encounters issues.

In [ ]:
# Download and extract dataset
print("\n📦 Downloading dataset from Borealis...")
public_doi = "doi:10.5683/SP3/H3HGWF"
dataset_files = get_public_dataset_info(public_doi)

if dataset_files:
    for file_info in dataset_files:
        if file_info['filename'].endswith('.zip'):
            print(f"📥 Downloading {file_info['filename']}...")
            downloaded_zip = download_public_file(file_info['file_id'])
            if downloaded_zip:
                print("📂 Extracting files...")
                unzip_file(downloaded_zip, "./")
            break

# Find where images are located
print("\n🔍 Locating image files...")
image_directory = find_image_directory()

if not image_directory:
    print("❌ ERROR: Could not find image directory")
    print("Current directory contents:")
    for item in os.listdir('.'):
        print(f"  {item}")
    exit()

print(f"✅ Found image directory: {image_directory}")

## Image Processing Configuration & Detection Loop

This is the core processing loop that handles each image individually. For each image, it loads and converts it to RGB format, applies the `transform` to create a `tensor`, runs inference through the model, and processes the results. The `confidence_threshold` of `0.5` filters out low-confidence detections. For valid detections, it draws red bounding boxes and adds labels with confidence scores. Each annotated image is saved to the organized output directory with detailed progress reporting.

In [ ]:
# Process images
print(f"\n🎯 Processing images from: {image_directory}")
print(f"📁 Saving results to: {annotated_dir}")

# Set confidence threshold for detection
confidence_threshold = 0.5
transform = F.to_tensor

# Find all JPG files
image_files = glob.glob(os.path.join(image_directory, '*.JPG'))
print(f"📷 Found {len(image_files)} images to process")

if not image_files:
    print("❌ No JPG files found!")
    exit()

# Process each image
annotated_count = 0
detected_count = 0

for i, image_path in enumerate(image_files, 1):
    print(f"\n[{i}/{len(image_files)}] Processing: {os.path.basename(image_path)}")

    try:
        # Load and process image
        image = Image.open(image_path).convert('RGB')
        image_tensor = transform(image)

        with torch.no_grad():
            prediction = model([image_tensor])

        boxes = prediction[0]['boxes']
        scores = prediction[0]['scores']
        labels = prediction[0]['labels']

        # Create figure
        fig, ax = plt.subplots(1, figsize=(12, 8))
        ax.imshow(image)
        ax.set_title(f'Detected Objects: {os.path.basename(image_path)}', fontsize=14)

        # Count detections above threshold
        detections_in_image = 0

        for box, score, label in zip(boxes, scores, labels):
            if score > confidence_threshold:
                detections_in_image += 1
                print(f"  🎯 Detection: Label {label.item()}, Score: {score.item():.3f}")

                # Draw bounding box
                x, y, xmax, ymax = box
                rect = patches.Rectangle((x, y), (xmax - x), (ymax - y),
                                       linewidth=2, edgecolor='red', facecolor='none')
                ax.add_patch(rect)

                # Add label text
                ax.text(x, y-5, f'Label: {label.item()}\nScore: {score.item():.2f}',
                       bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.7),
                       fontsize=8)

        if detections_in_image > 0:
            detected_count += 1
            print(f"  ✅ Found {detections_in_image} objects above threshold")
        else:
            print(f"  ⚪ No objects detected above threshold")

        # Save annotated image
        image_name = os.path.basename(image_path)
        save_path = os.path.join(annotated_dir, f'annotated_{image_name}')

        plt.tight_layout()
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()

        annotated_count += 1
        print(f"  💾 Saved: annotated_{image_name}")

    except Exception as e:
        print(f"  ❌ Error processing {image_path}: {e}")
        continue

## Print Processing Results & Display Annotated Images

This final module provides comprehensive results analysis and visualization. It calculates and displays summary statistics including processing success rates and detection rates. The advanced visualization system creates a grid layout displaying all annotated images simultaneously, with 5 images per row for optimal viewing. The grid automatically adjusts for different numbers of images and handles edge cases gracefully.

In [ ]:
# Final summary
print("\n" + "=" * 50)
print("🎉 PROCESSING COMPLETE!")
print(f"📊 Summary:")
print(f"   • Total images processed: {annotated_count}")
print(f"   • Images with detections: {detected_count}")
print(f"   • Detection rate: {(detected_count/annotated_count*100):.1f}%" if annotated_count > 0 else "   • No images processed")
print(f"   • Results saved in: {annotated_dir}")
print(f"   • Total files in output folder: {len(os.listdir(annotated_dir))}")

# List some example output files
output_files = os.listdir(annotated_dir)
if output_files:
    print(f"\n📁 Example output files:")
    for i, filename in enumerate(output_files[:5]):
        print(f"   {i+1}. {filename}")
    if len(output_files) > 5:
        print(f"   ... and {len(output_files) - 5} more files")

# Display annotated images
print(f"\n🖼️ Displaying all {len(output_files)} annotated images...")
print("=" * 50)

# Display all annotated images in a grid
if output_files:
    # Set fixed grid: 5 images per row for better visibility
    num_images = len(output_files)
    cols = 5  # Fixed at 5 columns
    rows = math.ceil(num_images / cols)

    # Create a large figure with bigger images (6 inches per image width, 4 inches height)
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 6, rows * 4))
    fig.suptitle(f'All {num_images} Annotated Images - Animal Detection Results (5 per row)', fontsize=20, y=0.98)

    # Handle case where we have only one row or column
    if rows == 1:
        axes = axes.reshape(1, -1)
    elif cols == 1:
        axes = axes.reshape(-1, 1)

    # Display each image
    for idx, filename in enumerate(sorted(output_files)):
        row = idx // cols
        col = idx % cols

        try:
            img_path = os.path.join(annotated_dir, filename)
            img = Image.open(img_path)

            axes[row, col].imshow(img)
            axes[row, col].set_title(filename.replace('annotated_', '').replace('.JPG', ''), fontsize=10)
            axes[row, col].axis('off')

        except Exception as e:
            print(f"Error loading {filename}: {e}")
            axes[row, col].set_title(f"Error: {filename}", fontsize=10)
            axes[row, col].axis('off')

    # Hide any unused subplots
    for idx in range(num_images, rows * cols):
        row = idx // cols
        col = idx % cols
        axes[row, col].axis('off')

    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    plt.show()

    print(f"✅ Successfully displayed all {num_images} annotated images!")
else:
    print("❌ No annotated images found to display.")

print(f"\n✅ All done! Check the '{output_dir}' folder in your file browser.")